<a href="https://colab.research.google.com/github/gsrit31/agentic_ai/blob/main/LLM_Function_Calling_after_gemini_auto_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')
os.environ['GEMINI_API_KEY'] = api_key

In [4]:
import json
import os
from typing import List

from litellm import completion

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"


tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]

# Our rules are simplified since we don't have to worry about getting a specific output format
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.
"""
}]

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

messages = agent_rules + memory

response = completion(
        model="gemini/gemini-2.0-flash",
    messages=messages,
    tools=tools,
    max_tokens=1024
)

print(response) # Print the full response

# Extract the tool call from the response, note we don't have to parse now!
if response.choices[0].message.tool_calls:
  tool = response.choices[0].message.tool_calls[0]
  tool_name = tool.function.name
  tool_args = json.loads(tool.function.arguments)
  result = tool_functions[tool_name](**tool_args)

  print(f"Tool Name: {tool_name}")
  print(f"Tool Arguments: {tool_args}")
  print(f"Result: {result}")
else:
  print("No tool call was made by the model.")

What would you like me to do? tell me list of files in abc directory
ModelResponse(id='4QJgaLK8K9ST1dkP1M7jCA', created=1751122657, model='gemini-2.0-flash', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='I am sorry, I cannot access directories. I can only list files. Would you like to list the files?\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=24, prompt_tokens=80, total_tokens=104, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=80, image_tokens=None)), vertex_ai_grounding_metadata=[], vertex_ai_url_context_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])
No tool call was made by the model.
